## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_1_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_2_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output_question, pooled_output_question = bert_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = bert_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.946295,0.583042,0.230807,0.527776,0.516622,0.536959,0.725162,0.690160,0.663220,...,0.907294,0.926349,0.572785,0.971986,0.971074,0.796190,0.012486,0.009092,0.853736,0.920319
1,46,0.883647,0.491128,0.005910,0.772697,0.808666,0.941776,0.570877,0.511705,0.089575,...,0.665403,0.939487,0.642016,0.973252,0.981998,0.890904,0.938543,0.135025,0.066093,0.888942
2,70,0.923692,0.684913,0.026993,0.731246,0.913025,0.954669,0.627434,0.529795,0.164590,...,0.896051,0.940380,0.612900,0.973317,0.976196,0.852252,0.012237,0.019440,0.887008,0.892427
3,132,0.925911,0.444748,0.005952,0.742861,0.716603,0.928639,0.553694,0.444070,0.060880,...,0.772536,0.945824,0.699978,0.961861,0.973479,0.912085,0.696548,0.120085,0.712696,0.907006
4,200,0.945859,0.485657,0.030181,0.899299,0.812257,0.949738,0.643519,0.661448,0.121222,...,0.654877,0.913744,0.650332,0.971758,0.960075,0.797839,0.264951,0.145381,0.523525,0.917455


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.906272,0.591780,0.040347,0.731037,0.816734,0.864099,0.592171,0.506024,0.242108,...,0.804971,0.939275,0.666048,0.969698,0.972476,0.871698,0.508613,0.123931,0.523830,0.908757
std,2812.670060,0.040497,0.152938,0.075646,0.107597,0.110527,0.125499,0.057499,0.099435,0.235676,...,0.090567,0.031620,0.052769,0.014634,0.017459,0.052003,0.333268,0.061701,0.301897,0.028857
min,39.000000,0.777833,0.266480,0.002489,0.221719,0.378352,0.196186,0.476483,0.324435,0.005188,...,0.547687,0.744246,0.496208,0.867325,0.807039,0.627261,0.003492,0.006691,0.020849,0.720976
25%,2572.000000,0.878906,0.464947,0.007034,0.673297,0.773179,0.842498,0.548410,0.431430,0.057148,...,0.733419,0.926122,0.633549,0.964771,0.967085,0.845444,0.156634,0.077324,0.247595,0.896393
50%,5093.000000,0.904916,0.569869,0.011754,0.737589,0.834939,0.910034,0.580711,0.476533,0.143847,...,0.802353,0.946839,0.662020,0.972731,0.977650,0.884306,0.587702,0.128162,0.523640,0.915031
75%,7482.000000,0.937258,0.716496,0.030380,0.796278,0.892724,0.938908,0.622750,0.560555,0.381804,...,0.890149,0.960638,0.694382,0.978983,0.983047,0.908837,0.819576,0.163948,0.806812,0.928647
max,9640.000000,0.982797,0.926294,0.588122,0.974841,0.987945,0.983645,0.754045,0.799136,0.943633,...,0.965157,0.985845,0.837042,0.989996,0.994640,0.963008,0.960171,0.313193,0.990201,0.961804
